In [1]:
import data
import string
from collections import OrderedDict
from functools import partial

abc = string.ascii_lowercase

class Dance:
    def __init__(self, programs, instructions_text):
        self.programs = list(programs)
        self.parse(instructions_text)
        self.cache_ = OrderedDict()
        self.hit_cache = 0
        self.make_cache()
        
    def make_cache(self):
        self.loop_size = 0
        while self.get_cached() == False:
            self.follow_instructions()
            self.loop_size += 1
        self.cache_list = list(self.cache_.items())
    
    def parse(self, instructions_text):
        instructions = []
        for line in instructions_text.split(','):
            instruction = line[0]
            values = line[1:]
            if instruction == 's':
                a = int(values)
                instructions.append(partial(self.spin, a))
            elif instruction == 'x':
                a, b = values.split('/')
                a, b = int(a), int(b)
                instructions.append(partial(self.exchange, a, b))
            elif instruction == 'p':
                a, b = values.split('/')
                instructions.append(partial(self.partner, a, b))
        self.instructions = instructions
        
    def follow_instructions(self):
        start_name = str(self)
        cached = self.get_cached()
        if cached:
            self.hit_cache += 1
            self.programs = cached
            return
        
        for instruction in self.instructions:
            instruction()
            
        self.cache(start_name)
            
    def get_cached(self):
        name = str(self)
        if name in self.cache_:
            return self.cache_[name].copy()
        else:
            return False
        
    def get_nth(self, n):
        key, value = self.cache_list[(n-1) % self.loop_size]
        return ''.join(value)
        
    def cache(self, start_name):
        self.cache_[start_name] = self.programs.copy()
        
    def spin(self, x):
        self.programs = self.programs[-x:] + self.programs[:-x]
        
    def exchange(self, a, b):
        self.programs[a], self.programs[b] = self.programs[b], self.programs[a]
    
    def partner(self, a, b):
        a_idx = self.programs.index(a)
        b_idx = self.programs.index(b)

        self.programs[a_idx], self.programs[b_idx] = b, a
    
    def __str__(self):
        return ''.join(self.programs)

    
d = Dance(abc[:5], data.test_data)
d.follow_instructions()
assert str(d) == 'baedc'

d = Dance(abc[:16], data.data)


print('answer 1')
print(d.get_nth(1))

print('answer 2')
print(d.get_nth(int(1e9)))

answer 1
ehdpincaogkblmfj
answer 2
bpcekomfgjdlinha


In [2]:
def run():
    d = Dance(abc[:16], data.data)
    d.get_nth(int(1e9))

%timeit run()

403 ms ± 15.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
